In [ ]:
%load_ext autoreload
%matplotlib inline

In [79]:
from sklearn.model_selection import cross_validate, ShuffleSplit, KFold, RepeatedKFold
from sklearn.metrics import make_scorer
from datetime import datetime
from joblib import Parallel, delayed
from collections import namedtuple
import numpy as np
import scipy

In [23]:
%autoreload 2
from datasets import make_datasets

In [24]:
datasets = make_datasets(year=False)

In [71]:
%autoreload 2
from metrics import normal_nll, rmse, mae, auc_rmse, auc_mae

In [75]:
%autoreload 2
from shallow_models import LinearRegression, BayesianLinearRegression, GBTQuantile, RFUncertainty, XGBaseline, XGBLogLikelihood
models = [RFUncertainty, LinearRegression, BayesianLinearRegression, GBTQuantile, XGBaseline, XGBLogLikelihood]

In [80]:
Results = namedtuple('Results', 'datetime dataset model shape normal_nll rmse mae auc_rmse auc_mae')

In [77]:
results = []

for d, (X,y) in datasets.items():
    try:
        X = X.values
        y = y.values
    except AttributeError:
        pass
    
    if d == 'year':
        cv = ShuffleSplit(1, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=3)
    elif d.startswith('make'):
        cv = KFold(n_splits=2)
    else:
        cv = RepeatedKFold(n_splits=5, n_repeats=1)
    
    for m in models:
        reg = m()
        cv_metrics = []
        for train_index, test_index in cv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            reg.fit(X, y)
            pred_mean, pred_std = reg.predict(X)
            cv_metrics.append((
                normal_nll(y, pred_mean, pred_std),
                rmse(y, pred_mean),
                mae(y, pred_mean),
                auc_rmse(y, pred_mean, pred_std),
                auc_mae(y, pred_mean, pred_std)))

        metrics_mean = np.mean(cv_metrics, axis = 1)
        metrics_stderr = scipy.stats.sem(cv_metrics, axis = 1)
        
        r = Results(
                str(datetime.now()),
                d, 
                m.__name__,
                X.shape,
                *zip(metrics_mean, metrics_stderr)
            )
        results.append(r)
        print(r)

Results(datetime='2018-03-01 19:57:22.695867', dataset='boston', model='RFUncertainty', shape=(506, 13), normal_nll=(2.455614031620532e+57, 2.4556140316205316e+57), rmse=(4.066076679841884e+57, 4.066076679841884e+57), mae=(5.71920948616602e+57, 5.7192094861660194e+57), auc_rmse=(2.216876086956517e+57, 2.2168760869565172e+57), auc_mae=(2.1445731225296325e+57, 2.144573122529632e+57))
Results(datetime='2018-03-01 19:57:23.174432', dataset='boston', model='LinearRegression', shape=(506, 13), normal_nll=(3.869981139822862, 0.33501465337831454), rmse=(3.869981139822862, 0.33501465337831454), mae=(3.869981139822862, 0.33501465337831454), auc_rmse=(3.869981139822862, 0.33501465337831454), auc_mae=(3.869981139822862, 0.33501465337831454))
Results(datetime='2018-03-01 19:57:23.638335', dataset='boston', model='BayesianLinearRegression', shape=(506, 13), normal_nll=(3.987720908492345, 0.4216856005732957), rmse=(3.987720908492345, 0.4216856005732957), mae=(3.987720908492345, 0.4216856005732957), a

KeyboardInterrupt: 

In [ ]:
def eval_dataset_model(d, X, y):
    try:
        X = X.values
        y = y.values
    except AttributeError:
        pass
    
    if d == 'year':
        cv = ShuffleSplit(1, test_size=0.1)
    elif d == 'protein':
        cv = KFold(n_splits=3)
    elif d.startswith('make'):
        cv = KFold(n_splits=2)
    else:
        cv = RepeatedKFold(n_splits=5, n_repeats=1)
    
    for m in models:
        reg = m()
        cv_metrics = []
        for train_index, test_index in cv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            reg.fit(X, y)
            pred_mean, pred_std = reg.predict(X)
            cv_metrics.append((
                normal_nll(y, pred_mean, pred_std),
                rmse(y, pred_mean),
                mae(y, pred_mean),
                auc_rmse(y, pred_mean, pred_std),
                auc_mae(y, pred_mean, pred_std)))

        metrics_mean = np.mean(cv_metrics, axis = 1)
        metrics_stderr = scipy.stats.sem(cv_metrics, axis = 1)
        
        r = Results(
            str(datetime.now()),
            d, 
            m.__name__,
            X.shape,
            *zip(metrics_mean, metrics_stderr)
        )
        print(r)
    return r

par_results = Parallel(n_jobs=16)(delayed(eval_dataset_model)(d, X, y) for d, (X, y) in datasets.items())

Results(datetime='2018-02-28 21:24:21.068409', dataset='boston', model='LinearRegression', shape=(506, 13), normal_nll=(3.869981139822862, 0.33501465337831454), rmse=(3.869981139822862, 0.33501465337831454), mae=(3.869981139822862, 0.33501465337831454), auc_rmse=(3.869981139822862, 0.33501465337831454), auc_mae=(3.869981139822862, 0.33501465337831454))
Results(datetime='2018-02-28 21:24:21.169216', dataset='yacht', model='LinearRegression', shape=(308, 6), normal_nll=(7.057690588006613, 0.9092927174428592), rmse=(7.057690588006613, 0.9092927174428592), mae=(7.057690588006613, 0.9092927174428592), auc_rmse=(7.057690588006613, 0.9092927174428592), auc_mae=(7.057690588006613, 0.9092927174428592))
Results(datetime='2018-02-28 21:24:22.710413', dataset='energy', model='LinearRegression', shape=(768, 8), normal_nll=(2.7577124067912324, 0.23897518027914125), rmse=(2.7577124067912324, 0.23897518027914125), mae=(2.7577124067912324, 0.23897518027914125), auc_rmse=(2.7577124067912324, 0.238975180

In [18]:
scipy.stats.norm.cdf(-1)

0.15865525393145707

In [58]:
X,y=datasets['boston']

In [64]:
from sklearn.ensemble import RandomForestRegressor
rf =  RandomForestRegressor(n_estimators=10)
rf.fit(X,y)
pred_mean = rf.predict(X)
percentile = scipy.stats.norm.cdf(-1) # One Gaussian std
err_down = []
err_up = []
dt_pred = np.vstack([dt.predict(X) for dt in rf.estimators_])
err_down = np.percentile(dt_pred, 100*percentile, axis=0)
err_up = np.percentile(dt_pred, 100*(1-percentile), axis=0)
pred_std = (err_up - err_down)/2

for i in range(len(X)):
    preds = []
    for pred in rf.estimators_:
        preds.append(pred.predict(X[i:i+1])[0])
    err_down.append(np.percentile(preds, 100*percentile))
    err_up.append(np.percentile(preds, 100*(1-percentile)))
pred_std = (np.array(err_up) - np.array(err_down))/2

In [70]:
np.percentile(np.vstack([dt.predict(X) for dt in rf.estimators_]),100*percentile, axis=0)

array([23.5995281 , 21.6       , 34.7       , 33.4       , 33.4       ,
       28.7       , 18.9       , 19.31347674, 15.08463566, 16.54278973,
       15.        , 18.9       , 19.88557946, 19.9       , 18.2       ,
       19.9       , 20.48557946, 17.5       , 20.2       , 18.2       ,
       13.1       , 17.5       , 13.84231783, 14.5       , 15.6       ,
       13.9       , 16.6       , 14.8       , 18.4       , 19.62695348,
       12.7       , 14.5       , 12.91394864, 13.1       , 13.5       ,
       18.4995281 , 19.37068701, 20.77115891, 20.28369186, 28.81394864,
       33.75579457, 26.6       , 25.08557946, 24.52836919, 20.68510756,
       19.3       , 19.47115891, 16.6       , 14.4       , 19.4       ,
       19.7       , 20.5       , 25.        , 23.4       , 17.88510756,
       33.28557946, 24.7       , 28.41347674, 22.67068701, 19.6       ,
       18.7       , 16.        , 22.2       , 23.78557946, 32.54231783,
       22.5       , 19.34278973, 21.02742539, 17.4       , 19.68

In [81]:
import pickle
with open('par_results.pkl', 'rb') as f:
    pr = pickle.load(f)

In [83]:
import json
json.dumps(pr)

'[["2018-03-01 20:00:27.904705", "boston", "LinearRegression", [506, 13], [2.9621311462524305, 1.4802973661668753e-16], [4.679506300635516, 0.0], [3.272944637996936, 0.0], [4.506588688366446, 2.9605947323337506e-16], [3.9287349258629805, 2.9605947323337506e-16]], ["2018-03-01 20:00:31.722094", "concrete", "LinearRegression", [1030, 8], [3.7562737659149485, 1.4802973661668753e-16], [10.353609808895646, 5.921189464667501e-16], [8.214343706221813, 0.0], [12.67012757796323, 5.921189464667501e-16], [10.959921770889947, 5.921189464667501e-16]], ["2018-03-01 20:00:30.481193", "energy", "LinearRegression", [768, 8], [2.4859556592703806, 1.4802973661668753e-16], [2.906696247412608, 0.0], [2.050870442737076, 1.4802973661668753e-16], [3.4906226888427936, 1.4802973661668753e-16], [2.8544169956933034, 1.4802973661668753e-16]], ["2018-03-01 20:10:15.380734", "kin8nm", "LinearRegression", [8192, 8], [-0.18155005466952676, 9.25185853854297e-18], [0.2017978978983736, 9.25185853854297e-18], [0.162078160

In [85]:
models

[shallow_models.RFUncertainty,
 shallow_models.LinearRegression,
 shallow_models.BayesianLinearRegression,
 shallow_models.GBTQuantile,
 shallow_models.XGBaseline,
 shallow_models.XGBLogLikelihood]

In [87]:
from itertools import product

In [92]:
[(d, m) for (d, (X, y)), m in product(datasets.items(), models)]

[('boston', shallow_models.RFUncertainty),
 ('boston', shallow_models.LinearRegression),
 ('boston', shallow_models.BayesianLinearRegression),
 ('boston', shallow_models.GBTQuantile),
 ('boston', shallow_models.XGBaseline),
 ('boston', shallow_models.XGBLogLikelihood),
 ('concrete', shallow_models.RFUncertainty),
 ('concrete', shallow_models.LinearRegression),
 ('concrete', shallow_models.BayesianLinearRegression),
 ('concrete', shallow_models.GBTQuantile),
 ('concrete', shallow_models.XGBaseline),
 ('concrete', shallow_models.XGBLogLikelihood),
 ('energy', shallow_models.RFUncertainty),
 ('energy', shallow_models.LinearRegression),
 ('energy', shallow_models.BayesianLinearRegression),
 ('energy', shallow_models.GBTQuantile),
 ('energy', shallow_models.XGBaseline),
 ('energy', shallow_models.XGBLogLikelihood),
 ('kin8nm', shallow_models.RFUncertainty),
 ('kin8nm', shallow_models.LinearRegression),
 ('kin8nm', shallow_models.BayesianLinearRegression),
 ('kin8nm', shallow_models.GBTQuant